In [74]:
from qiskit import QuantumCircuit
from qiskit.dagcircuit import DAGCircuit
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.tools.visualization import dag_drawer

In [350]:
qc = QuantumCircuit(4,4)
qc.x(0)
qc.cx(0,1)
qc.cx(2,3)
qc.x(0)
qc.x(1)
qc.cx(0,1)
qc.cx(2,3)
qc.cx(1,2)
qc.measure(range(4), range(4))

In [351]:
qc.draw()

┌───┐     ┌───┐          ┌─┐      
q_0: ┤ X ├──■──┤ X ├──■───────┤M├──────
     └───┘┌─┴─┐├───┤┌─┴─┐     └╥┘┌─┐   
q_1: ─────┤ X ├┤ X ├┤ X ├──■───╫─┤M├───
          └───┘└───┘└───┘┌─┴─┐ ║ └╥┘┌─┐
q_2: ──■────■────────────┤ X ├─╫──╫─┤M├
     ┌─┴─┐┌─┴─┐ ┌─┐      └───┘ ║  ║ └╥┘
q_3: ┤ X ├┤ X ├─┤M├────────────╫──╫──╫─
     └───┘└───┘ └╥┘            ║  ║  ║ 
c: 4/════════════╩═════════════╩══╩══╩═
                 3             0  1  2

In [368]:
layers, measurements = circuit_to_benchmark_layers(qc)
render_layers(layers, measurements).draw()

┌───┐      ░ ┌───┐      ░       ░ ┌─┐         
q_0: ┤ X ├──■───░─┤ X ├──■───░───────░─┤M├─────────
     └───┘┌─┴─┐ ░ ├───┤┌─┴─┐ ░       ░ └╥┘┌─┐      
q_1: ─────┤ X ├─░─┤ X ├┤ X ├─░───■───░──╫─┤M├──────
          └───┘ ░ └───┘└───┘ ░ ┌─┴─┐ ░  ║ └╥┘┌─┐   
q_2: ──■────────░───■────────░─┤ X ├─░──╫──╫─┤M├───
     ┌─┴─┐      ░ ┌─┴─┐      ░ └───┘ ░  ║  ║ └╥┘┌─┐
q_3: ┤ X ├──────░─┤ X ├──────░───────░──╫──╫──╫─┤M├
     └───┘      ░ └───┘      ░       ░  ║  ║  ║ └╥┘
c: 4/═══════════════════════════════════╩══╩══╩══╩═
                                        0  1  2  3

In [364]:
def circuit_to_benchmark_layers(qc):
    layers = []
    #remove final measurements and store as separate layer
    measurements = qc.copy_empty_like()
    for inst in qc:
        if inst.operation.name == "measure":
            measurements.append(inst)

    #convert qc to a list of instructions
    inst_list = [inst for inst in qc if inst.operation.name != "measure"] 

    #iterate through instructions adding to layers according to the rules until all are added
    while inst_list:
        circ = qc.copy_empty_like()
        layer_qubits = set()

        for inst in inst_list.copy():
            if not layer_qubits.intersection(inst.qubits):
                if len(inst.qubits) == 2:
                    layer_qubits = layer_qubits.union(inst.qubits)
                circ.append(inst)
                inst_list.remove(inst)

        layers.append(circ)

    return layers, measurements

In [367]:
def render_layers(layers, measurements):
    qc = layers[0].copy_empty_like()
    for l in layers:
        qc = qc.compose(l)
        qc.barrier()
    qc = qc.compose(measurements)
    return qc